#A Tale of a Reduction

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to exercise reductions a lot and we want to choose the best container for doing this.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.031 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 30.887 MiB


and choose a different container for the data that we want to reduce, starting with a list:

##Regular lists

In [2]:
a = [i for i in range(1000*1000)]

In [2] used 31.250 MiB RAM in 0.141s, peaked 0.000 MiB above current, total RAM usage 62.137 MiB


Now, proceed with a simple reduction (sum):

In [20]:
t = %timeit -o sum(a)

The slowest run took 4.74 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 6.94 ms per loop
In [20] used 0.004 MiB RAM in 2.975s, peaked 0.000 MiB above current, total RAM usage 129.383 MiB


which, in MIPS (Mega-Instructions-Per-Second) is:

In [24]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 86.2)
In [24] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 129.387 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

In [5]:
# Create an array of 'l'ong integers (8 bytes on 32-bit platforms)
import array
arr = array.array('l', a)

In [5] used 7.465 MiB RAM in 0.102s, peaked 0.000 MiB above current, total RAM usage 69.629 MiB


7.7 MB vs 31 MB seems like a good deal.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [6]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

7.827456

In [6] used 0.039 MiB RAM in 0.020s, peaked 0.000 MiB above current, total RAM usage 69.668 MiB


But how it performs during reductions?

In [22]:
t = %timeit -o sum(arr)

100 loops, best of 3: 11.6 ms per loop
In [22] used 0.004 MiB RAM in 4.895s, peaked 0.000 MiB above current, total RAM usage 129.387 MiB


In [23]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 86.185)
In [23] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 129.387 MiB


Well, that's a bit disappointing, as the array object performs a 60% slower than a regular array.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

##NumPy

In [9]:
import numpy as np
na = np.array(a, dtype=np.int64)

In [9] used 22.293 MiB RAM in 0.145s, peaked 0.000 MiB above current, total RAM usage 91.984 MiB


In [10]:
%timeit sum(na)

10 loops, best of 3: 84.1 ms per loop
In [10] used 0.031 MiB RAM in 3.574s, peaked 0.000 MiB above current, total RAM usage 92.016 MiB


In [25]:
print("MIPS:", round(1e6 / 83.7e-3 / 1e6, 3))

('MIPS:', 11.947)
In [25] used 0.004 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 129.391 MiB


Oops, this is more than 7x slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [26]:
t = %timeit -o na.sum()

1000 loops, best of 3: 548 µs per loop
In [26] used 0.000 MiB RAM in 2.380s, peaked 0.000 MiB above current, total RAM usage 129.391 MiB


In [27]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 1824.734)
In [27] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 129.391 MiB


This is more than 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.  

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

##Using compressed in-memory containers with bcolz

In [28]:
import bcolz

In [28] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 129.391 MiB


In [29]:
ca = bcolz.carray(na)

In [29] used 0.250 MiB RAM in 0.010s, peaked 0.000 MiB above current, total RAM usage 129.641 MiB


Why so much memory consumption?  This is an artifact of the OS memory subsystem and is probably OS dependent.  Let's try again and create a new carray:

In [30]:
ca2 = bcolz.carray(na)

In [30] used 0.254 MiB RAM in 0.011s, peaked 0.000 MiB above current, total RAM usage 129.895 MiB


In [31]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.25390625)
In [31] used 0.004 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 129.898 MiB


Ok, this time the amount of memory used seems much lower.  Let's see how much memory the container thinks it has:

In [18]:
ca

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 394.28 KB; ratio: 19.81
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [18] used 0.125 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 129.289 MiB


In [34]:
t = %timeit -o ca.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

The slowest run took 5.19 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 2.03 ms per loop
('MIPS:', 493.082)
In [34] used 0.004 MiB RAM in 0.982s, peaked 0.000 MiB above current, total RAM usage 129.906 MiB


This is around 4x slower than a regular NumPy array, but the size of the array is just a 20x smaller.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [36]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [36] used 7.215 MiB RAM in 0.095s, peaked 0.000 MiB above current, total RAM usage 137.121 MiB


In [37]:
cau2 = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [37] used 7.988 MiB RAM in 0.102s, peaked 0.000 MiB above current, total RAM usage 145.109 MiB


In [38]:
cau

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [38] used 0.004 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 145.113 MiB


In [43]:
t = %timeit -o cau.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 2.05 ms per loop
('MIPS:', 487.469)
In [43] used 0.000 MiB RAM in 1.037s, peaked 0.000 MiB above current, total RAM usage 145.141 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one.  On the other hand, carray is not competitive with NumPy, at least for this kind of operations. 

In [44]:
%timeit sum(cau)

10 loops, best of 3: 30.8 ms per loop
In [44] used 0.027 MiB RAM in 1.782s, peaked 0.000 MiB above current, total RAM usage 145.168 MiB


### Exercise: Using bcolz in real scenarios

bcolz does not get good results only with synthetic data, but with real data too.  Be sure to check out this URL:

http://nbviewer.ipython.org/gist/alimanfoo/e93a532eb6bde311ea39/genotype_bitshuffle.ipynb

and let's discuss this specific case of bcolz usage in genomics.